In [ ]:
!pip install ydata_profiling pythainlp

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.3/19.3 MB 58.8 MB/s eta 0:00:00


In [ ]:
!pip install pythainlp ydata_profiling

  Using cached pythainlp-5.1.2-py3-none-any.whl.metadata (8.0 kB)
Using cached pythainlp-5.1.2-py3-none-any.whl (19.3 MB)


In [3]:
import pandas as pd
import numpy as np
from collections import Counter
import re
import os
import logging
from ydata_profiling import ProfileReport
from functools import partial
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin

import matplotlib.pyplot as plt
import matplotlib as mpl


In [4]:
mpl.rcParams['font.family'] = 'Noto Sans Thai'  # or 'Tahoma', 'TH Sarabun New'

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    handlers=[logging.StreamHandler()]
)
logger = logging.getLogger('thai_attractions_processor')


In [7]:
class ThaiTextPreprocessor:
    """Class to handle Thai text preprocessing with graceful fallback"""

    def __init__(self):
        self.PYTHAINLP_AVAILABLE = False
        self.thai_stopwords_list = []

        try:
            from pythainlp.tokenize import word_tokenize
            from pythainlp.corpus import thai_stopwords
            self.PYTHAINLP_AVAILABLE = True
            self.word_tokenize = word_tokenize
            self.thai_stopwords_list = list(thai_stopwords())
            # Add some common Thai punctuation/symbols
            custom_removal_list = ['"', 'ๆ', '(', ')', ':', '-', '.', ',', "'", ' ']
            self.thai_stopwords_list.extend(custom_removal_list)
            self.thai_stopwords_list = list(set(self.thai_stopwords_list)) # Ensure uniqueness
            logger.info("PyThaiNLP successfully loaded")
        except ImportError:
            logger.warning("PyThaiNLP not installed. Text preprocessing will be basic.")

    def preprocess(self, text):
        """Preprocess Thai text with tokenization and stopword removal"""
        if not self.PYTHAINLP_AVAILABLE:
            if isinstance(text, str):
                text = text.lower()
                text = re.sub(r'[^\u0E00-\u0E7F\sA-Za-z0-9]', '', text)
                return text.split()
            return []

        if not isinstance(text, str):
            return []

        # Clean text
        text = re.sub(r'http[s]?://\S+', '', text)
        text = re.sub(r'\S+@\S+', '', text)
        text = re.sub(r'[<>/\'"""''\(\)\[\]{}!?:;]', ' ', text)
        text = re.sub(r'(\d+)\.(\d+)', r'\1punct\2', text)
        text = re.sub(r'[^\u0E00-\u0E7F\sA-Za-z0-9_]', '', text)
        text = text.replace('punct', '.')
        text = text.strip()

        # Tokenize and filter
        words = self.word_tokenize(text, engine='newmm')
        processed_words = [
            word for word in words
            if word not in self.thai_stopwords_list and len(word) > 1
            and not word.isspace() and not word.isnumeric()
        ]
        return processed_words

<>:37: SyntaxWarning: invalid escape sequence '\('
<>:37: SyntaxWarning: invalid escape sequence '\('
/var/folders/zc/6vdm7g3s70q83l9rmg6pcc8m0000gn/T/ipykernel_47483/1661847630.py:37: SyntaxWarning: invalid escape sequence '\('
  text = re.sub(r'[<>/\'"""''\(\)\[\]{}!?:;]', ' ', text)


In [9]:
# prompt: list all uniqe of  categories and Sub-categories

# Assuming your dataframe is named 'df' and has columns 'Category' and 'Sub-Category'
df = pd.read_csv('allattractions.csv')
unique_categories = df['ATTR_CATAGORY_TH'].unique()
unique_subcategories = df['ATTR_SUB_TYPE_TH'].unique()

print("Unique Categories:")
print(unique_categories)

print("\nUnique Sub-Categories:")
unique_subcategories


Unique Categories:
['แหล่งท่องเที่ยวทางธรรมชาติ'
 'แหล่งท่องเที่ยวทางประวัติศาสตร์ และวัฒนธรรม'
 'แหล่งท่องเที่ยวสำหรับกิจกรรมพิเศษ นันทนาการ และความสนใจพิเศษ']

Unique Sub-Categories:


array(['ภูเขา/ธรณีสัณฐานเฉพาะ', 'เขื่อน/อ่างเก็บน้ำ', 'ศูนย์หัตถกรรม',
       'ศาสนสถาน (วัด/โบสถ์/มัสยิด ฯลฯ)',
       'สวนสัตว์/ศูนย์ฝึกสัตว์/พิพิธภัณฑ์สัตว์', 'อ่าว/หาดทราย/ชายทะเล',
       'ทุ่งดอกไม้และพืชพันธุ์', 'จุดชมวิว',
       'ชุมชนโบราณ/โบราณสถาน/โบราณวัตถุ',
       'ศูนย์วิจัย (เกษตร)/สถานีทดลอง (เกษตร)', 'หมู่เกาะ',
       'วิถีชีวิตความเป็นอยู่ (ชุมชน)', 'พิพิธภัณฑ์',
       'ศูนย์กีฬา/สนามกีฬาสถานที่ทางการกีฬาทางบก/ทางน้ำ/ทางอากาศ',
       'น้ำตก', 'ห้างสรรพสินค้า/แหล่งช้อปปิ้ง/ตลาดสด/ตลาดนัด/ถนนคนเดิน',
       'ถ้ำ',
       'ศูนย์การเรียนรู้ฯ (เกี่ยวกับกิจกรรม ผลิตภัณฑ์ และภูมิปัญญาในท้องถิ่น)',
       'ศูนย์ศึกษาธรรมชาติ/พิพิธภัณธ์ธรรมชาติ (พืชพันธุ์)',
       'ไร่/สวนเกษตร (ฟาร์มสัตว์/ประมง)', 'แม่น้ำ/ลำคลอง/แก่ง',
       'ทะเลสาบ/หนอง/บึง', 'ประวัติความเป็นมา', 'ธีมปาร์ค (Theme Park)',
       'โรงละคร/โรงมหรสพ (โชว์)', 'อุทยานแห่งชาติ', 'ตลาดน้ำ/ตลาดโบราณ',
       'จุดผ่านแดน/ชายแดน', 'กำแพงเมือง/คูเมือง', 'สวนสาธารณะ/สวนหย่อม',
       'พระตำหนัก/วัง/พระราชวัง', 'โ

In [10]:
class SeasonalClassifier:
    """Class to determine suitable seasons for attractions"""

    def __init__(self, text_preprocessor):
        self.text_preprocessor = text_preprocessor

        # Mapping dictionaries
        self.seasonal_keywords = {
            'winter': [
                'หนาว', 'เย็น', 'ฤดูหนาว', 'ลมหนาว', 'หมอก', 'ดอกไม้เมืองหนาว', 'เคาท์ดาวน์', 'ปีใหม่',
                'พฤศจิกายน', 'ธันวาคม', 'มกราคม', 'กุมภาพันธ์', 'เทศกาลคริสต์มาส', 'คริสต์มาส',
                'ภูกระดึง', 'ดอย', 'ภู', 'ขึ้นดอย', 'ชมดาว', 'แม่คะนิ้ง', 'ซากุระเมืองไทย', 'นางพญาเสือโคร่ง',
                'ปลายฝนต้นหนาว', 'ลอยกระทง'
            ],
            'summer': [
                'ร้อน', 'ฤดูร้อน', 'แดด', 'ทะเล', 'ชายหาด', 'เกาะ', 'ดำน้ำ', 'สงกรานต์', 'ว่ายน้ำ',
                'มีนาคม', 'เมษายน', 'พฤษภาคม', 'เที่ยวทะเล', 'หาดทราย', 'อากาศร้อน', 'คลายร้อน', 'เมษา'
            ],
            'rainy': [
                'ฝน', 'ฤดูฝน', 'หน้าฝน', 'น้ำตก', 'เขียว', 'เขียวขจี', 'ชุ่มชื้น', 'พรรษา', 'เข้าพรรษา',
                'มิถุนายน', 'กรกฎาคม', 'สิงหาคม', 'กันยายน', 'ตุลาคม', 'ปลูกป่า', 'ล่องแก่ง', 'ทุ่งดอกกระเจียว',
                'ดูเห็ด', 'ปลายฝนต้นหนาว', 'น้ำหลาก'
            ]
        }

        self.month_to_season_map = {
            1: 'winter', 2: 'winter', 3: 'summer', 4: 'summer', 5: 'summer',
            6: 'rainy', 7: 'rainy', 8: 'rainy', 9: 'rainy', 10: 'rainy',
            11: 'winter', 12: 'winter'
        }

        self.thai_month_names = {
            'มกราคม': 1, 'กุมภาพันธ์': 2, 'มีนาคม': 3, 'เมษายน': 4, 'พฤษภาคม': 5, 'มิถุนายน': 6,
            'กรกฎาคม': 7, 'สิงหาคม': 8, 'กันยายน': 9, 'ตุลาคม': 10, 'พฤศจิกายน': 11, 'ธันวาคม': 12,
            'ม.ค.': 1, 'ก.พ.': 2, 'มี.ค.': 3, 'เม.ย.': 4, 'พ.ค.': 5, 'มิ.ย.': 6,
            'ก.ค.': 7, 'ส.ค.': 8, 'ก.ย.': 9, 'ต.ค.': 10, 'พ.ย.': 11, 'ธ.ค.': 12,
            'มกรา': 1, 'กุมภา': 2, 'มีนา': 3, 'เมษา': 4, 'พฤษภา': 5, 'มิถุนา': 6,
            'กรกฎา': 7, 'สิงหา': 8, 'กันยา': 9, 'ตุลา': 10, 'พฤศจิ': 11, 'ธันวา': 12
        }

        # Category-based heuristics
        self.category_season_heuristic_map = {
            # Main categories
            'แหล่งท่องเที่ยวธรรมชาติประเภททะเล/หมู่เกาะ/ชายหาด': ['summer', 'winter'],
            'แหล่งท่องเที่ยวธรรมชาติประเภทเกาะ': ['summer', 'winter'],
            'แหล่งท่องเที่ยวธรรมชาติประเภทชายหาด': ['summer', 'winter'],
            'แหล่งท่องเที่ยวธรรมชาติประเภทแหลม': ['summer', 'winter'],
            'แหล่งท่องเที่ยวธรรมชาติประเภทอ่าว': ['summer', 'winter'],
            'แหล่งท่องเที่ยวธรรมชาติประเภทถ้ำ': ['winter', 'summer', 'rainy'],
            'แหล่งท่องเที่ยวธรรมชาติประเภทน้ำตก': ['rainy', 'winter'],
            'แหล่งท่องเที่ยวธรรมชาติประเภทภูเขา': ['winter', 'rainy'],
            'แหล่งท่องเที่ยวธรรมชาติประเภทอุทยานแห่งชาติ': ['winter', 'summer', 'rainy'],
            'แหล่งท่องเที่ยวธรรมชาติประเภทวนอุทยาน': ['winter', 'summer', 'rainy'],
            'แหล่งท่องเที่ยวธรรมชาติประเภทจุดชมทิวทัศน์': ['winter', 'rainy'],
            'แหล่งท่องเที่ยวธรรมชาติประเภทบ่อน้ำร้อน': ['winter', 'rainy'],
            'แหล่งท่องเที่ยวธรรมชาติประเภทแก่ง/คันคุ้งน้ำ': ['rainy', 'summer'],
            'แหล่งท่องเที่ยวประเภทวัด': ['winter', 'summer', 'rainy'],
            'แหล่งท่องเที่ยวประเภทตลาด': ['winter', 'summer', 'rainy'],
            'แหล่งท่องเที่ยวทางประวัติศาสตร์': ['winter', 'summer', 'rainy'],
            'แหล่งท่องเที่ยวประเภทชุมชน': ['winter', 'summer', 'rainy'],
            'แหล่งท่องเที่ยวประเภทพิพิธภัณฑ์': ['winter', 'summer', 'rainy'],
            'แหล่งท่องเที่ยวเพื่อนันทนาการ': ['winter', 'summer', 'rainy'],
            'แหล่งท่องเที่ยวเชิงเกษตร': ['winter', 'rainy'],
            # Sub-categories
            'อุทยานแห่งชาติทางทะเล': ['summer', 'winter'],
            'จุดชมวิว ทิวทัศน์': ['winter', 'rainy'],
            'เส้นทางศึกษาธรรมชาติ': ['winter', 'rainy'],
            # Add more as needed based on data profiling results
            'ตลาดน้ำ': ['winter', 'summer', 'rainy'],
            'ภูเขา/ธรณีสัณฐานเฉพาะ': ['winter', 'rainy'],
            'เขื่อน/อ่างเก็บน้ำ': ['rainy', 'winter'],
            'ศูนย์หัตถกรรม': ['winter', 'summer', 'rainy'],
            'ศาสนสถาน (วัด/โบสถ์/มัสยิด ฯลฯ)': ['winter', 'summer', 'rainy'],
            'สวนสัตว์/ศูนย์ฝึกสัตว์/พิพิธภัณฑ์สัตว์': ['winter', 'summer'],
            'อ่าว/หาดทราย/ชายทะเล': ['summer', 'winter'],
            'ทุ่งดอกไม้และพืชพันธุ์': ['winter'],
            'จุดชมวิว': ['winter', 'rainy'],
            'ชุมชนโบราณ/โบราณสถาน/โบราณวัตถุ': ['winter', 'summer'],
            'ศูนย์วิจัย (เกษตร)/สถานีทดลอง (เกษตร)': ['winter', 'rainy'],
            'หมู่เกาะ': ['summer', 'winter'],
            'วิถีชีวิตความเป็นอยู่ (ชุมชน)': ['winter', 'summer', 'rainy'],
            'พิพิธภัณฑ์': ['winter', 'summer', 'rainy'],
            'ศูนย์กีฬา/สนามกีฬาสถานที่ทางการกีฬาทางบก/ทางน้ำ/ทางอากาศ': ['winter', 'summer'],
            'น้ำตก': ['rainy', 'winter'],
            'ห้างสรรพสินค้า/แหล่งช้อปปิ้ง/ตลาดสด/ตลาดนัด/ถนนคนเดิน': ['winter', 'summer', 'rainy'],
            'ถ้ำ': ['winter', 'rainy', 'summer'],
            'ศูนย์การเรียนรู้ฯ (เกี่ยวกับกิจกรรม ผลิตภัณฑ์ และภูมิปัญญาในท้องถิ่น)': ['winter', 'summer', 'rainy'],
            'ศูนย์ศึกษาธรรมชาติ/พิพิธภัณธ์ธรรมชาติ (พืชพันธุ์)': ['winter', 'rainy'],
            'ไร่/สวนเกษตร (ฟาร์มสัตว์/ประมง)': ['winter', 'rainy'],
            'แม่น้ำ/ลำคลอง/แก่ง': ['rainy', 'summer'],
            'ทะเลสาบ/หนอง/บึง': ['rainy', 'winter'],
            'ประวัติความเป็นมา': ['winter', 'summer'],
            'ธีมปาร์ค (Theme Park)': ['winter', 'summer'],
            'โรงละคร/โรงมหรสพ (โชว์)': ['winter', 'summer', 'rainy'],
            'อุทยานแห่งชาติ': ['winter', 'summer', 'rainy'],
            'ตลาดน้ำ/ตลาดโบราณ': ['winter', 'summer', 'rainy'],
            'จุดผ่านแดน/ชายแดน': ['winter', 'summer'],
            'กำแพงเมือง/คูเมือง': ['winter', 'summer'],
            'สวนสาธารณะ/สวนหย่อม': ['winter', 'summer'],
            'พระตำหนัก/วัง/พระราชวัง': ['winter', 'summer'],
            'โครงการหลวง/โครงการพระราชดำริ': ['winter', 'rainy'],
            'เขตรักษาพันธุ์สัตว์ป่า': ['winter', 'rainy'],
            'พรุ/ป่าชายเลน/พื้นที่ชุ่มน้ำ': ['rainy', 'winter'],
            'น้ำพุร้อน/บ่อน้ำร้อน/ธารน้ำ': ['winter', 'rainy'],
            'วนอุทยาน': ['winter', 'summer', 'rainy'],
            'สวนสนุก/สวนน้ำ': ['summer', 'winter'],
            'สถานปฏิบัติธรรม': ['winter', 'summer', 'rainy'],
            'หมู่บ้าน': ['winter', 'summer', 'rainy'],
            'แหล่งปะการังน้ำลึก/น้ำตื้น': ['summer', 'winter'],
            'อนุสาวรีย์/อนุสรณ์สถาน': ['winter', 'summer'],
            'ศูนย์วัฒนธรรม': ['winter', 'summer', 'rainy'],
            'สุขภาพและความงาม': ['winter', 'summer', 'rainy'],
            'สวนพฤกษศาสตร์': ['winter', 'rainy'],
            'ศูนย์ศึกษาประวัติศาสตร์': ['winter', 'summer'],
            'อุทยานประวัติศาสตร์': ['winter', 'summer'],
            'เหมือง/ประวัติศาสตร์การทำเหมือง': ['winter', 'summer'],
            'ศูนย์ประชุมฯ': ['winter', 'summer'],
            'เขตห้ามล่าสัตว์ป่า': ['winter', 'rainy'],
            'ซากฟอสซิล': ['winter', 'summer'],
            'สวนรุกชาติ': ['winter', 'rainy'],
            'โรงภาพยนต์โบราณ (โรงหนังเก่า)': ['winter', 'summer', 'rainy'],
            'งานประเพณี': ['winter', 'summer', 'rainy'],

        }

    def parse_start_end_dates(self, date_text):
        """Parse start-end dates to identify seasons"""
        seasons_found = set()
        if not isinstance(date_text, str) or date_text.strip() == '-' or date_text.strip() == '':
            return list(seasons_found)

        # Keep original date_text for some phrase matching, normalized for others
        original_date_text_lower = date_text.lower()

        all_year_phrases = ["ทุกวัน", "ตลอดทั้งปี", "เปิดทุกวัน", "ทุกเดือน", "ตลอดปี", "ทั้งปี", "all year", "daily", "0000-00-00", "ไม่กำหนด"]
        closed_rainy_phrases = ["ปิดฤดูฝน", "ปิดหน้าฝน", "ยกเว้นฤดูฝน", "งดท่องเที่ยวฤดูฝน", "closed during rainy season"]

        is_all_year = any(phrase in original_date_text_lower for phrase in all_year_phrases)
        is_closed_rainy = any(phrase in date_text for phrase in closed_rainy_phrases)

        if is_all_year:
            if is_closed_rainy:
                return ['summer', 'winter']
            return ['winter', 'summer', 'rainy']

        if is_closed_rainy:
            return ['summer', 'winter']

        found_month_numbers = set()
        for thai_month, month_num in self.thai_month_names.items():
            if thai_month in date_text:
                found_month_numbers.add(month_num)

        if found_month_numbers:
            if len(found_month_numbers) >= 2:
                min_month = min(found_month_numbers)
                max_month = max(found_month_numbers)
                # Handle wrap-around (e.g., Nov - Feb)
                if (min_month > max_month and (min_month - max_month) < 10) or \
                   (max_month - min_month > 5 and max_month - min_month < 10):
                    for m_val in range(min_month, 13): seasons_found.add(self.month_to_season_map[m_val])
                    for m_val in range(1, max_month + 1): seasons_found.add(self.month_to_season_map[m_val])
                else: # Normal range
                    for m_val in range(min_month, max_month + 1): seasons_found.add(self.month_to_season_map[m_val])
            elif len(found_month_numbers) == 1:
                seasons_found.add(self.month_to_season_map[list(found_month_numbers)[0]])

        # Check for explicit season mentions
        if "ฤดูหนาว" in date_text or "หน้าหนาว" in date_text: seasons_found.add('winter')
        if "ฤดูร้อน" in date_text or "หน้าร้อน" in date_text: seasons_found.add('summer')
        if "ฤดูฝน" in date_text or "หน้าฝน" in date_text:
             if not is_closed_rainy: seasons_found.add('rainy')

        if "ปลายฝนต้นหนาว" in date_text:
            seasons_found.add('rainy'); seasons_found.add('winter')

        return list(seasons_found) if seasons_found else []

    def assign_season_from_text(self, att_name, detail, activity, remark, suitable_duration_text):
        """Extract seasons from text fields"""
        combined_text_tokens = []
        text_fields_to_process = [att_name, detail, activity, remark, suitable_duration_text]
        for text_field in text_fields_to_process:
            if isinstance(text_field, str):
                combined_text_tokens.extend(self.text_preprocessor.preprocess(text_field))

        if not combined_text_tokens: return []

        season_scores = Counter()
        for token in combined_text_tokens:
            for season, keywords in self.seasonal_keywords.items():
                if token in keywords:
                    season_scores[season] += 1

        # Give more weight to seasons mentioned in suitable_duration_text
        if isinstance(suitable_duration_text, str):
            sdt_lower = suitable_duration_text
            if "ฤดูหนาว" in sdt_lower or "หน้าหนาว" in sdt_lower: season_scores['winter'] += 5
            if "ฤดูร้อน" in sdt_lower or "หน้าร้อน" in sdt_lower: season_scores['summer'] += 5
            if "ฤดูฝน" in sdt_lower or "หน้าฝน" in sdt_lower: season_scores['rainy'] += 5
            if "ปลายฝนต้นหนาว" in sdt_lower: season_scores['rainy'] += 3; season_scores['winter'] += 3
            if "ตลอดปี" in sdt_lower or "ทุกฤดู" in sdt_lower or "ทั้งปี" in sdt_lower:
                return ['winter', 'summer', 'rainy']

            duration_tokens = self.text_preprocessor.preprocess(suitable_duration_text)
            for token in duration_tokens:
                for season, keywords in self.seasonal_keywords.items():
                    if token in keywords: season_scores[season] += 2

        return [season for season, score in season_scores.items() if score > 0]

    def apply_category_heuristics(self, row, current_seasons):
        """Apply category-based heuristics to determine seasons"""
        if not current_seasons or current_seasons == ['unknown']:
            pass
        else:
            return current_seasons

        category = str(row['ATTR_CATAGORY_TH']).strip()
        sub_category = str(row['ATTR_SUB_TYPE_TH']).strip()

        # Check sub-category first for more specific rules
        if sub_category and sub_category in self.category_season_heuristic_map:
            return self.category_season_heuristic_map[sub_category]

        if category and category in self.category_season_heuristic_map:
            return self.category_season_heuristic_map[category]

        # Partial match as fallback
        if isinstance(sub_category, str):
             for key, seasons in self.category_season_heuristic_map.items():
                 if key in sub_category: return seasons
        if isinstance(category, str):
            for key, seasons in self.category_season_heuristic_map.items():
                if key in category: return seasons

        return current_seasons

    def determine_suitable_season(self, row):
        """Main method to determine suitable season for an attraction"""
        name_th = row['ATT_NAME_TH']
        detail_th = row['ATT_DETAIL_TH']
        activity = row['ATT_ACTIVITY']
        start_end = row['ATT_START_END']
        suitable_duration = row['ATT_SUITABLE_DURATION']
        remark = row['ATT_REMARK']

        # Initialize seasons
        final_seasons = set()

        # 1. Direct check for "all year" in suitable_duration
        if isinstance(suitable_duration, str) and any(p in suitable_duration for p in ["ตลอดปี", "ทุกฤดู", "ทั้งปี"]):
            return ['rainy', 'summer', 'winter']

        # 2. Parse ATT_START_END
        seasons_from_dates = self.parse_start_end_dates(start_end)

        # 3. Analyze text fields
        seasons_from_text = self.assign_season_from_text(name_th, detail_th, activity, remark, suitable_duration)

        # Combine results
        final_seasons.update(seasons_from_dates)
        final_seasons.update(seasons_from_text)

        # Handle special case
        if 'all_year' in final_seasons:
            return ['rainy', 'summer', 'winter']

        # Convert set to list
        current_result = sorted(list(final_seasons)) if final_seasons else []

        # 4. Apply category heuristics if no season found yet
        if not current_result:
            heuristic_seasons = self.apply_category_heuristics(row, [])
            if heuristic_seasons and heuristic_seasons != ['unknown']:
                current_result = sorted(list(set(heuristic_seasons)))

        return current_result if current_result else ['unknown']


In [14]:
class MissingValueHandler(BaseEstimator, TransformerMixin):
    """Handle missing values in the DataFrame"""

    def __init__(self):
        pass

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        """Fill missing values appropriately"""
        df = X.copy()

        text_cols = ['ATT_NAME_TH', 'ATT_DETAIL_TH', 'ATT_ACTIVITY', 'ATT_START_END',
                     'ATT_SUITABLE_DURATION', 'ATT_REMARK', 'ATTR_CATAGORY_TH', 'ATTR_SUB_TYPE_TH']

        for col in text_cols:
            if col in df.columns:
                df[col] = df[col].fillna('')
            else:
                logger.warning(f"Column '{col}' not found in DataFrame. Creating empty column.")
                df[col] = ''

        return df

In [15]:
class SeasonalTagger(BaseEstimator, TransformerMixin):
    """Add seasonal tags to the DataFrame"""

    def __init__(self, text_preprocessor=None):
        self.text_preprocessor = text_preprocessor or ThaiTextPreprocessor()
        self.seasonal_classifier = SeasonalClassifier(self.text_preprocessor)

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        """Add seasonal tags to DataFrame"""
        df = X.copy()

        logger.info("Applying seasonal tagging...")
        df['SUITABLE_SEASON'] = df.apply(self.seasonal_classifier.determine_suitable_season, axis=1)

        # Apply fallback for 'unknown' values
        unknown_mask = df['SUITABLE_SEASON'].apply(lambda x: x == ['unknown'] or x == [])
        if unknown_mask.sum() > 0:
            logger.info(f"Setting {unknown_mask.sum()} 'unknown' values to 'all seasons'")
            df.loc[unknown_mask, 'SUITABLE_SEASON'] = pd.Series(
    [['rainy', 'summer', 'winter']] * unknown_mask.sum(),
    index=df[unknown_mask].index
)

        return df

In [20]:
class AttractionDataProcessor:
    """Main class to handle the entire data processing pipeline"""

    def __init__(self, input_file):
        self.input_file = input_file
        self.df = None
        self.text_preprocessor = ThaiTextPreprocessor()
        self.profile = None

        # Define the pipeline
        self.pipeline = Pipeline([
            ('missing_handler', MissingValueHandler()),
            ('seasonal_tagger', SeasonalTagger(self.text_preprocessor))
        ])

    def load_data(self):
        """Load data from CSV file"""
        try:
            self.df = pd.read_csv(self.input_file)
            logger.info(f"Successfully loaded dataset with shape: {self.df.shape}")
            return True
        except FileNotFoundError:
            logger.error(f"Error: '{self.input_file}' not found.")
            self.df = pd.DataFrame()
            return False

    def generate_profile_report(self, output_path='data_profile_report.html', minimal=False):
        """Generate a profile report using ydata-profiling"""
        if self.df is None or self.df.empty:
            logger.error("Cannot generate profile report: DataFrame is empty")
            return False

        logger.info("Generating profile report (this may take some time)...")

        profile_kwargs = {
            'title': 'Thai Attractions Dataset Profile',
            'minimal': minimal
        }

        if minimal:
            # Add configurations for minimal report
            profile_kwargs.update({
                'explorative': True,
                'correlations': None
            })

        self.profile = ProfileReport(self.df, **profile_kwargs)
        self.profile.to_file(output_path)

        logger.info(f"Profile report generated and saved to {output_path}")
        return True

    def process_data(self):
        """Run the processing pipeline"""
        if self.df is None or self.df.empty:
            logger.error("Cannot process data: DataFrame is empty")
            return None

        logger.info("Running data processing pipeline...")
        processed_df = self.pipeline.transform(self.df)
        logger.info("Pipeline processing complete")

        return processed_df

    def get_season_statistics(self, df=None):
        """Get statistics about seasonal distribution"""
        target_df = df if df is not None else self.df

        if target_df is None or target_df.empty or 'SUITABLE_SEASON' not in target_df.columns:
            logger.error("Cannot get statistics: DataFrame is empty or missing SUITABLE_SEASON column")
            return None

        logger.info("Calculating seasonal statistics...")

        # Convert season lists to strings for counting
        season_str_counts = target_df['SUITABLE_SEASON'].astype(str).value_counts()

        # Count occurrence of each season (handling lists)
        season_counts = Counter()
        for seasons in target_df['SUITABLE_SEASON']:
            if isinstance(seasons, list):
                for season in seasons:
                    season_counts[season] += 1

        # Multi-season combinations
        multi_season_counts = target_df['SUITABLE_SEASON'].apply(
            lambda x: tuple(sorted(x)) if isinstance(x, list) else tuple([x])
        ).value_counts()

        return {
            'individual_season_counts': dict(season_counts),
            'season_combinations': {str(k): v for k, v in multi_season_counts.items()},
            'unknown_count': len(target_df[target_df['SUITABLE_SEASON'].apply(lambda x: x == ['unknown'])])
        }

    def run_full_analysis(self, output_dir='.', generate_profile=True):
        """Run the full analysis pipeline"""
        success = self.load_data()
        if not success:
            return None

        # Generate profile report
        if generate_profile:
            profile_path = os.path.join(output_dir, 'thai_attractions_profile.html')
            self.generate_profile_report(profile_path)

        # Process data
        processed_df = self.process_data()

        # Generate statistics
        stats = self.get_season_statistics(processed_df)

        # Save processed data
        output_path = os.path.join(output_dir, 'allattractions_with_season.csv')
        processed_df.to_csv(output_path, index=False)
        logger.info(f"Processed data saved to {output_path}")

        return {
            'processed_df': processed_df,
            'statistics': stats
        }

In [21]:
from sklearn import set_config
set_config(display='diagram')

# Run the processor
processor = AttractionDataProcessor('allattractions.csv')
results = processor.run_full_analysis()

if results:
    processed_df = results['processed_df']
    stats = results['statistics']

    print("\n---- Seasonal Statistics ----")
    print(f"Individual season counts: {stats['individual_season_counts']}")

    print("\nTop 5 season combinations:")
    for combo, count in list(stats['season_combinations'].items())[:5]:
        print(f"  {combo}: {count}")

    print("\n---- Sample of Processed Data ----")
    sample_cols = ['ATT_NAME_TH', 'ATTR_CATAGORY_TH', 'SUITABLE_SEASON']
    print(processed_df[sample_cols].sample(min(5, len(processed_df))))

    # 🧠 Display the pipeline visually
    print("\n---- Pipeline Diagram ----")
    processor.pipeline

2025-05-11 04:39:11,355 - thai_attractions_processor - INFO - PyThaiNLP successfully loaded
2025-05-11 04:39:11,592 - thai_attractions_processor - INFO - Successfully loaded dataset with shape: (8242, 31)
2025-05-11 04:39:11,592 - thai_attractions_processor - INFO - Generating profile report (this may take some time)...


Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

100%|██████████| 31/31 [00:01<00:00, 21.71it/s]


Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

/opt/homebrew/anaconda3/lib/python3.12/site-packages/ydata_profiling/visualisation/utils.py:73: UserWarning: Glyph 3616 (\N{THAI CHARACTER PHO SAMPHAO}) missing from font(s) Arial.
  plt.savefig(
/opt/homebrew/anaconda3/lib/python3.12/site-packages/ydata_profiling/visualisation/utils.py:73: UserWarning: Glyph 3634 (\N{THAI CHARACTER SARA AA}) missing from font(s) Arial.
  plt.savefig(
/opt/homebrew/anaconda3/lib/python3.12/site-packages/ydata_profiling/visualisation/utils.py:73: UserWarning: Glyph 3588 (\N{THAI CHARACTER KHO KHWAI}) missing from font(s) Arial.
  plt.savefig(
/opt/homebrew/anaconda3/lib/python3.12/site-packages/ydata_profiling/visualisation/utils.py:73: UserWarning: Glyph 3585 (\N{THAI CHARACTER KO KAI}) missing from font(s) Arial.
  plt.savefig(
/opt/homebrew/anaconda3/lib/python3.12/site-packages/ydata_profiling/visualisation/utils.py:73: UserWarning: Glyph 3621 (\N{THAI CHARACTER LO LING}) missing from font(s) Arial.
  plt.savefig(
/opt/homebrew/anaconda3/lib/python3

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

2025-05-11 04:39:23,263 - thai_attractions_processor - INFO - Profile report generated and saved to ./thai_attractions_profile.html
2025-05-11 04:39:23,264 - thai_attractions_processor - INFO - Running data processing pipeline...
2025-05-11 04:39:23,329 - thai_attractions_processor - INFO - Applying seasonal tagging...
2025-05-11 04:39:33,947 - thai_attractions_processor - INFO - Pipeline processing complete
2025-05-11 04:39:33,950 - thai_attractions_processor - INFO - Calculating seasonal statistics...
2025-05-11 04:39:34,131 - thai_attractions_processor - INFO - Processed data saved to ./allattractions_with_season.csv



---- Seasonal Statistics ----
Individual season counts: {'rainy': 6582, 'winter': 7277, 'summer': 6929}

Top 5 season combinations:
  ('rainy', 'summer', 'winter'): 5462
  ('summer', 'winter'): 916
  ('rainy', 'winter'): 593
  ('summer',): 438
  ('rainy',): 414

---- Sample of Processed Data ----
                                          ATT_NAME_TH  \
4149                                 ชุมชนบ้านวัดเกต    
4727                                 บ้านช่างทำหัวโขน   
6381  เขตห้ามล่าสัตว์ป่าหนองปลักพระยาและเขาระยาบังสา    
1359            ศาลสมเด็จพระเจ้าตากสินมหาราช จันทบุรี   
5641                                      หาดบ้านแก้ง   

                                 ATTR_CATAGORY_TH          SUITABLE_SEASON  
4149  แหล่งท่องเที่ยวทางประวัติศาสตร์ และวัฒนธรรม  [rainy, summer, winter]  
4727  แหล่งท่องเที่ยวทางประวัติศาสตร์ และวัฒนธรรม  [rainy, summer, winter]  
6381                   แหล่งท่องเที่ยวทางธรรมชาติ  [rainy, summer, winter]  
1359  แหล่งท่องเที่ยวทางประวัติศาสตร์ และวัฒนธรรม 

In [22]:
processor.pipeline

Pipeline(steps=[('missing_handler', MissingValueHandler()),
                ('seasonal_tagger',
                 SeasonalTagger(text_preprocessor=<__main__.ThaiTextPreprocessor object at 0x3314e13a0>))])

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]


100%|██████████| 31/31 [00:05<00:00,  5.20it/s]


Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/ydata_profiling/visualisation/utils.py:73: UserWarning: Glyph 3616 (\N{THAI CHARACTER PHO SAMPHAO}) missing from font(s) DejaVu Sans.
  plt.savefig(
/usr/local/lib/python3.11/dist-packages/ydata_profiling/visualisation/utils.py:73: UserWarning: Glyph 3634 (\N{THAI CHARACTER SARA AA}) missing from font(s) DejaVu Sans.
  plt.savefig(
/usr/local/lib/python3.11/dist-packages/ydata_profiling/visualisation/utils.py:73: UserWarning: Glyph 3588 (\N{THAI CHARACTER KHO KHWAI}) missing from font(s) DejaVu Sans.
  plt.savefig(
/usr/local/lib/python3.11/dist-packages/ydata_profiling/visualisation/utils.py:73: UserWarning: Glyph 3585 (\N{THAI CHARACTER KO KAI}) missing from font(s) DejaVu Sans.
  plt.savefig(
/usr/local/lib/python3.11/dist-packages/ydata_profiling/visualisation/utils.py:73: UserWarning: Glyph 3621 (\N{THAI CHARACTER LO LING}) missing from font(s) DejaVu Sans.
  plt.savefig(
/usr/local/lib/python3.11/dist-packages/ydata_profiling/visualisation/

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/sklearn/pipeline.py:62: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(



---- Seasonal Statistics ----
Individual season counts: {'rainy': 7399, 'summer': 7416, 'winter': 7311}

Top 5 season combinations:
  ('rainy', 'summer', 'winter'): 6781
  ('summer',): 438
  ('rainy',): 414
  ('winter',): 287
  ('rainy', 'winter'): 125

---- Sample of Processed Data ----
                              ATT_NAME_TH  \
5693                       วนอุทยานเขาพาง   
7231  โครงการส่งเสริมศิลปาชีพ บ้านเวินบึก   
230      พิพิธภัณฑ์อัญมณีและเครื่องประดับ   
8064  วัดกร่าง จ.ปทุมธานี(หลวงพ่อนรสิงห์)   
1184                ถ้ำค้างคาวเขาช่องพราน   

                                       ATTR_CATAGORY_TH  \
5693                         แหล่งท่องเที่ยวทางธรรมชาติ   
7231  แหล่งท่องเที่ยวสำหรับกิจกรรมพิเศษ นันทนาการ แล...   
230         แหล่งท่องเที่ยวทางประวัติศาสตร์ และวัฒนธรรม   
8064        แหล่งท่องเที่ยวทางประวัติศาสตร์ และวัฒนธรรม   
1184                         แหล่งท่องเที่ยวทางธรรมชาติ   

              SUITABLE_SEASON  
5693  [rainy, summer, winter]  
7231  [rainy, summer